<a href="https://colab.research.google.com/github/SURESHBEEKHANI/Natural-Language-Processing/blob/main/Predict_Stock_Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


In [ ]:
data=pd.read_csv("/content/Data.csv", encoding = "ISO-8859-1")

In [ ]:
data.head()

In [ ]:
# Assuming 'Date' column is in datetime format or can be converted to it

# Step 1: Filter rows for training data where Date is before '2015-01-01'
train = data[data['Date'] < '2015-01-01']

# Step 2: Filter rows for testing data where Date is after '2014-12-31'
test = data[data['Date'] > '2014-12-31']

# Explanation:
# - 'train' DataFrame will contain all rows where Date is before January 1st, 2015.
# - 'test' DataFrame will contain all rows where Date is after December 31st, 2014.
#   (Note: This assumes '2014-12-31' is inclusive in 'test' as per the condition >, adjust as necessary)

# Ensure the 'Date' column in your dataset is in datetime format or parsed as such for accurate filtering.

In [ ]:
# Selecting columns 2 to 26 from the train DataFrame
data = train.iloc[:, 2:27]

# Removing all characters that are not letters (punctuations, numbers, etc.)
data.replace("[^a-zA-Z]", " ", regex=True, inplace=True)

# Creating a list of numbers from 0 to 24
list1 = [i for i in range(25)]

# Converting the list of numbers to strings
new_Index = [str(i) for i in list1]

# Renaming the columns of the data DataFrame
data.columns = new_Index

# Displaying the first 5 rows of the data DataFrame
data.head(5)


In [ ]:
# Convert all headlines in each column to lower case
for index in new_Index:
    data[index] = data[index].str.lower()

# Display the first row of the modified data DataFrame
data.head(1)


In [ ]:
# Join all values in the first row, from columns 0 to 24, into a single string
joined_text = ' '.join(str(x) for x in data.iloc[1, 0:25])

# Display the joined string
print(joined_text)


In [ ]:
# Initialize an empty list to store the combined text from each row
headlines = []

# Loop over each row in the data DataFrame
for row in range(0, len(data.index)):
    # Combine text from columns 0 to 24 into a single string for the current row
    combined_text = ' '.join(str(x) for x in data.iloc[row, 0:25])

    # Add the combined text to the headlines list
    headlines.append(combined_text)

# Display the first few combined headlines (optional)
print(headlines[:5])


In [ ]:
headlines[0]

In [ ]:
# Import necessary libraries
# CountVectorizer: Helps in converting text data into a matrix of token counts (Bag of Words)
from sklearn.feature_extraction.text import CountVectorizer
# RandomForestClassifier: A machine learning algorithm used for classification tasks
from sklearn.ensemble import RandomForestClassifier

# Implementing Bag of Words model
# CountVectorizer converts text data into a matrix of word counts, here we use pairs of words (bigrams)
countvector = CountVectorizer(ngram_range=(2, 2))

# Transform the headlines into a matrix of word counts
# This converts the list of headlines into numerical data that can be used by the machine learning algorithm
traindataset = countvector.fit_transform(headlines)

# Implementing Random Forest Classifier
# RandomForestClassifier is a machine learning algorithm used to classify data into categories
# n_estimators=200: The number of trees in the forest
# criterion='entropy': The function to measure the quality of a split
randomclassifier = RandomForestClassifier(n_estimators=200, criterion='entropy')

# Fit the RandomForest model with the training data
# randomclassifier.fit: Trains the Random Forest model using the transformed headlines (traindataset) and the labels from the training data
randomclassifier.fit(traindataset, train['Label'])

# After running this code, the Random Forest model will be trained and ready to make predictions


In [ ]:
# Predicting for the Test Dataset

# Initialize an empty list to store the transformed test headlines
test_transform = []

# Loop over each row in the test DataFrame
for row in range(0, len(test.index)):
    # Combine text from columns 2 to 26 into a single string for the current row
    combined_text = ' '.join(str(x) for x in test.iloc[row, 2:27])

    # Add the combined text to the test_transform list
    test_transform.append(combined_text)

# Transform the test headlines into a matrix of word counts
# This converts the list of test headlines into numerical data that can be used by the machine learning algorithm
test_dataset = countvector.transform(test_transform)

# Use the trained Random Forest model to make predictions on the test data
# The model will predict the category (label) for each headline in the test data
predictions = randomclassifier.predict(test_dataset)

# After running this code, the 'predictions' will contain the predicted labels for the test headlines


In [ ]:
# Import library to check accuracy
# These tools help us evaluate how well our model is performing
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Calculate the confusion matrix
# This matrix shows the number of correct and incorrect predictions made by the model
# Rows represent actual categories, and columns represent predicted categories
matrix = confusion_matrix(test['Label'], predictions)
print(matrix)

# Calculate the accuracy score
# This score tells us the percentage of correct predictions made by the model
score = accuracy_score(test['Label'], predictions)
print(score)

# Generate a detailed classification report
# This report provides precision, recall, and F1 score for each category
report = classification_report(test['Label'], predictions)
print(report)
